In [ ]:
# import relevant libraries

import pandas as pd
import numpy as np
import cv2
import os
import sklearn
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_curve, auc, precision_recall_curve, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, roc_auc_score, average_precision_score, mean_squared_error
import random
from PIL import Image
import math

In [ ]:
def krippendorff_alpha(preds, labels):
    # Calculate observed agreement
    obs_agree = np.sum(preds == labels) / len(preds)

    # Calculate expected agreement
    unique_labels = np.unique(labels)
    freq = np.zeros((len(unique_labels), len(preds)))
    for i, label in enumerate(unique_labels):
        freq[i] = preds == label
    exp_agree = np.sum(np.sum(freq, axis=0) * np.sum(freq, axis=0)) / (len(preds) * len(unique_labels))

    # Calculate alpha
    alpha = 1 - (1 - obs_agree) / (1 - exp_agree)
    
    return alpha

def auc_fn(predictions, labels):
    # convert preds and labs to numpy arrays
    preds = np.array(predictions)
    labs = np.array(labels)

    # calculate the one-hot encoding of the labels
    n_classes = len(np.unique(labs))
    one_hot_labs = np.zeros((len(labs), n_classes))
    one_hot_labs[np.arange(len(labs)), labs] = 1

    # calculate the ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(one_hot_labs[:, i], preds == i)
        roc_auc[i] = auc(fpr[i], tpr[i])

    # calculate the average AUC across all classes
    avg_auc = np.mean(list(roc_auc.values()))

    return avg_auc

def auc_pr_fn(preds, labs):
    n_classes = len(set(labs))
    auc_pr = []

    for i in range(n_classes):
        y_true = [1 if x == i else 0 for x in labs]
        y_score = [1 if x == i else 0 for x in preds]
        precision, recall, _ = precision_recall_curve(y_true, y_score)
        area = auc(recall, precision)
        auc_pr.append(area)
    
    auc_pr_mean = sum(auc_pr)/len(auc_pr)
    return auc_pr, auc_pr_mean

def sagr(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    y_true_sign = np.sign(y_true)
    y_pred_sign = np.sign(y_pred)
    sagr_value = np.mean(y_true_sign == y_pred_sign)
    return sagr_value

def concordance_correlation_coefficient(x, y):
    # Compute means and standard deviations of x and y
    x_mean, y_mean = np.mean(x), np.mean(y)
    x_std, y_std = np.std(x), np.std(y)
    
    # Compute covariance and correlation coefficient between x and y
    covariance = np.cov(x, y)[0][1]
    correlation_coefficient = covariance / (x_std * y_std)
    
    # Compute the mean square error between x and y
    mse_x = np.mean(np.square(x - x_mean))
    mse_y = np.mean(np.square(y - y_mean))
    mean_square_error = (mse_x + mse_y) / 2
    
    # Compute the concordance correlation coefficient
    concordance_correlation_coefficient = 2 * correlation_coefficient * np.sqrt(mse_x) * np.sqrt(mse_y) / \
        (np.square(x_std) + np.square(y_std) + np.square(x_mean - y_mean))
    
    return concordance_correlation_coefficient